In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.compute import ComputeManagementClient
from google.cloud import storage
import boto3


Setup Env & funcs

In [ ]:
# Define VM configurations
vm_configurations = [
    {
        'name': 'Standard_DS2_v2',
        'vcpus': 2,
        'memory': 7,
    },
    {
        'name': 'n1-standard-2',
        'vcpus': 2,
        'memory': 7.5,
    },
    {
        'name': 't2.medium',
        'vcpus': 2,
        'memory': 4,
    }
]

def get_azure_vm_price(vm_config):
    credential = DefaultAzureCredential()
    compute_client = ComputeManagementClient(credential, "your_subscription_id")
    
    vm_sizes = compute_client.virtual_machine_sizes.list("your_region")
    for size in vm_sizes:
        if size.name == vm_config['name']:
            return size.number_of_cores * size.number_of_cores * 0.01  # Simplified price calculation
    
    return None

def get_gcp_vm_price(vm_config):
    storage_client = storage.Client()
    skus = storage_client.list_blobs("cloud-pricing", prefix="CP-COMPUTEENGINE-VMIMAGE")
    
    for sku in skus:
        if vm_config['name'] in sku.name:
            return float(sku.download_as_text())
    
    return None

def get_aws_vm_price(vm_config):
    ec2_client = boto3.client('ec2', region_name='your_region')
    
    response = ec2_client.describe_instance_type_offerings(
        LocationType='availability-zone',
        Filters=[
            {
                'Name': 'instance-type',
                'Values': [vm_config['name']]
            }
        ]
    )
    
    if 'InstanceTypeOfferings' in response:
        price = response['InstanceTypeOfferings'][0]['PricingDetails'][0]['Price']
        return float(price)
    
    return None

Run the Tracker

In [ ]:
for config in vm_configurations:
    azure_price = get_azure_vm_price(config)
    gcp_price = get_gcp_vm_price(config)
    aws_price = get_aws_vm_price(config)
    
    if azure_price and gcp_price and aws_price:
        print(f"VM Configuration: {config['name']}")
        print(f"Azure Price: ${azure_price:.2f}")
        print(f"GCP Price: ${gcp_price:.2f}")
        print(f"AWS Price: ${aws_price:.2f}")
        print("=" * 30)
    else:
        print(f"Could not retrieve pricing information for {config['name']}")